# Predicting Check-ins of Foursquare Users in Tokyo

## 3 - Feature Engineering

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
# Load wrangled data.
df = pd.read_csv('df_afterwrangling.csv',
                 index_col=0,
                 parse_dates=[0],
                 date_parser=pd.to_datetime,
                 infer_datetime_format=True)

In [3]:
df.head()

,userid,venuecat,lat,long,day,hour,maincat,venuecat_encoded,maincat_encoded
timestamp,,,,,,,,,
2012-04-04 03:17:18,1541,Cosmetics Shop,35.705101,139.619590,2,3,Shop & Service,129,1
2012-04-04 03:22:04,868,Ramen / Noodle House,35.715581,139.800317,2,3,Food,2,2
2012-04-04 04:12:07,114,Convenience Store,35.714542,139.480065,2,4,Shop & Service,3,1
2012-04-04 04:12:13,868,Food & Drink Shop,35.725592,139.776633,2,4,Shop & Service,6,1
2012-04-04 04:18:23,1458,Housing Development,35.656083,139.734046,2,4,Residence,213,8


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 573703 entries, 2012-04-04 03:17:18 to 2013-02-16 11:35:29
Data columns (total 9 columns):
userid              573703 non-null int64
venuecat            573703 non-null object
lat                 573703 non-null float64
long                573703 non-null float64
day                 573703 non-null int64
hour                573703 non-null int64
maincat             573703 non-null object
venuecat_encoded    573703 non-null int64
maincat_encoded     573703 non-null int64
dtypes: float64(2), int64(5), object(2)
memory usage: 43.8+ MB


In [5]:
unique_venuecat = len(df.venuecat_encoded.unique())
start_date = '2012-Jul'

### Stacked Model, Level 0: Train Station, Subway, or Other

In [6]:
def create_time_window_cols(df, window_minutes):

    series_timestamps = df.index
    delta_minutes = datetime.timedelta(minutes=window_minutes)

    df['start_time'] = (series_timestamps - delta_minutes).time
    df['start_day'] = (series_timestamps - delta_minutes).weekday
    df['end_time'] = (series_timestamps + delta_minutes).time
    df['end_day'] = (series_timestamps + delta_minutes).weekday

    return(df)

In [7]:
def slice_by_time_window(df, start_time, start_day, end_time, end_day):

    series_times = df.index.time
    series_days = df.day

    if start_day == end_day:

        df_sliced = df.loc[(series_days == start_day) & (
            series_times > start_time) & (series_times < end_time), :]

    else:

        df_sliced = df.loc[(((series_days == start_day) & (series_times > start_time)) | (
            (series_days == end_day) & (series_times < end_time))), :]

    return(df_sliced)

In [8]:
def find_nearest_venues(lat_now, long_now, lat_all, long_all, venues, radius_meters):

    indices = ((long_now - long_all)**2 + (lat_now - lat_all) **
               2) <= (0.00000898 * radius_meters)**2  # 0.0000089 ~ 1 meter
    counts_nearby_venues = np.zeros(unique_venuecat)

    if any(indices):

        nearby_venues = venues[indices]

        for venue in nearby_venues:

            counts_nearby_venues[venue] += 1

    return(counts_nearby_venues)

In [9]:
def engineer_features(df, col, start_date, radius_meters, window_minutes=None, by_userid=False):

    # Create start and end times for timestamp of each row:

    if window_minutes:

        df = create_time_window_cols(df, window_minutes)

    datetime_start_date = datetime.datetime.strptime(start_date, "%Y-%b")
    nearby_venues = np.empty((df[start_date:].shape[0], unique_venuecat))

    row_num = 0

    for row in df.itertuples():

        if row.Index >= datetime_start_date:

            # Does not count current check-in as part of past history
            df_to_date = df[: row.Index].head(-1)

            if by_userid:

                df_to_date_userid = df_to_date.loc[df_to_date.userid ==
                                                   row.userid, :]

            else:

                df_to_date_userid = df_to_date

            if window_minutes:

                df_sliced = slice_by_time_window(
                    df_to_date_userid, row.start_time, row.start_day, row.end_time, row.end_day)

            else:

                df_sliced = df_to_date_userid

            if df_sliced.shape[0]:

                nearest_venues_found = find_nearest_venues(
                    row.lat, row.long, df_sliced.lat.values, df_sliced.long.values, df_sliced[col].values, radius_meters)

                nearby_venues[row_num] = nearest_venues_found

            else:

                nearby_venues[row_num] = np.zeros(unique_venuecat)

            row_num += 1

    df = pd.DataFrame(nearby_venues)

    df_normalized = df.div(df.sum(axis=1), axis=0).fillna(0)

    return(df_normalized)

In [10]:
df_creatures_of_habit = engineer_features(
    df, col='venuecat_encoded', start_date=start_date, radius_meters=100, window_minutes=60, by_userid=True)

In [11]:
df_creatures_of_habit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336029 entries, 0 to 336028
Columns: 247 entries, 0 to 246
dtypes: float64(247)
memory usage: 633.2 MB


In [12]:
df_creatures_of_habit.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df_creatures_of_habit.columns = [
    'userid_time_place_' + str(feature) for feature in range(unique_venuecat)]
df_creatures_of_habit.index = df[start_date:].index
df_creatures_of_habit.to_csv('df_creatures_of_habit.csv')

In [14]:
df_irregular_creatures_of_habit = engineer_features(
    df, col='venuecat_encoded', start_date=start_date, radius_meters=100, by_userid=True)

In [15]:
df_irregular_creatures_of_habit.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 336029 entries, 2012-07-01 04:12:45 to 2013-02-16 11:35:29
Columns: 247 entries, userid_place_0 to userid_place_246
dtypes: float64(247)
memory usage: 635.8 MB


In [16]:
df_irregular_creatures_of_habit.head()

,userid_place_0,userid_place_1,userid_place_2,userid_place_3,userid_place_4,userid_place_5,userid_place_6,userid_place_7,userid_place_8,userid_place_9,...,userid_place_237,userid_place_238,userid_place_239,userid_place_240,userid_place_241,userid_place_242,userid_place_243,userid_place_244,userid_place_245,userid_place_246
timestamp,,,,,,,,,,,,,,,,,,,,,
2012-07-01 04:12:45,0.0,0.0,0.00,0.0,0.0,1.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-07-01 04:13:52,0.0,0.0,0.00,0.0,0.0,1.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-07-01 04:16:11,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-07-01 04:37:30,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-07-01 04:43:24,0.0,0.0,0.16,0.0,0.0,0.84,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df_irregular_creatures_of_habit.columns = [
    'userid_place_' + str(feature) for feature in range(unique_venuecat)]
df_irregular_creatures_of_habit.index = df[start_date:].index
df_irregular_creatures_of_habit.to_csv('df_irregular_creatures_of_habit.csv')

In [18]:
df_whenever_wherever = pd.concat([df.userid, pd.get_dummies(df.venuecat_encoded, prefix="userid")], axis=1) \
    .groupby(["userid"]) \
    .expanding() \
    .mean() \
    .reset_index(level="userid", drop=True) \
    .sort_index() \
    .drop("userid", axis=1)

In [19]:
df_whenever_wherever.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 573703 entries, 2012-04-04 03:17:18 to 2013-02-16 11:35:29
Columns: 247 entries, userid_0 to userid_246
dtypes: float64(247)
memory usage: 1.1 GB


In [20]:
df_whenever_wherever.head()

,userid_0,userid_1,userid_2,userid_3,userid_4,userid_5,userid_6,userid_7,userid_8,userid_9,...,userid_237,userid_238,userid_239,userid_240,userid_241,userid_242,userid_243,userid_244,userid_245,userid_246
timestamp,,,,,,,,,,,,,,,,,,,,,
2012-04-04 03:17:18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-04-04 03:22:04,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-04-04 04:12:07,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-04-04 04:12:13,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-04-04 04:18:23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
df_whenever_wherever[start_date:].to_csv('df_whenever_wherever.csv')